In [1]:
using Random
using Base: Fix2
using Plots, LaTeXStrings, ColorSchemes
using FFTW
using CUDA

In [3]:
default(fontfamily="serif", legendfontsize=10, titlefontsize=12, palette=:seaborn_deep6)
gr()


msglen = 100
scheme = :qpsk

:qpsk

In [4]:
stream = bitrand(msglen) .|> Int
stream_d = CuArray(stream)

100-element CuArray{Int64, 1, CUDA.Mem.DeviceBuffer}:
 0
 0
 0
 1
 1
 0
 1
 1
 1
 1
 ⋮
 1
 0
 1
 0
 0
 1
 0
 1
 1

In [10]:
function cu_to_blocks(data::CuArray, type=:bpsk)
    nbits = Dict(:bpsk => 48, :qpsk => 96, :qam16 => 192, :qam64 => 288)
    pad_bits = nbits[type] - mod(length(data), nbits[type])
    data = vcat(data, zeros(pad_bits))

    # Reshape the ArrayFire array to the desired block size
    return reshape(data, nbits[type], :)
end

@info "To blocks"
t_to_blocks = @elapsed begin
    blocks_d = cu_to_blocks(stream_d, scheme)
end


┌ Info: To blocks
└ @ Main /home/icnpg/Escritorio/mat/ICNPG-IB/pll_tests.ipynb:10


0.030670321

In [11]:
blocks_d

96×2 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
 0.0  1.0
 0.0  0.0
 0.0  1.0
 1.0  1.0
 1.0  0.0
 0.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 ⋮    
 1.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 1.0  0.0
 0.0  0.0
 1.0  0.0
 0.0  0.0
 0.0  0.0

In [60]:
function ktoi_kernel(result, N, ks)
    for idx = 1:length(ks)
        @inbounds result[idx] = N[1] ÷ 16 * (ks[idx] % 16) + floor(ks[idx] ÷ 16) |> Int
    end
    return
end

function itoj_kernel(result, N, s, is)
    for idx = 1:length(is)
        @inbounds result[idx] = s[1] * floor(is[idx] ÷ s[1]) + (is[idx] + N[1] - floor((16 * is[idx]) ÷ N[1])) % s[1] |> Int
    end
    return
end

function cu_ktoi(N, ks)
    result_cuda = CUDA.CuArray{Int}(undef, N)
    @cuda threads=1 blocks=1 ktoi_kernel(result_cuda, CuArray([N]), ks)

    return result_cuda
end

function cu_itoj(N, s, is)
    result_cuda = CUDA.CuArray{Int}(undef, N)
    @cuda threads=1 blocks=1 itoj_kernel(result_cuda, CuArray([N]), CuArray([s]), is)

    return result_cuda
end

cu_itoj (generic function with 1 method)

In [61]:
N = 192

192

In [65]:
function cu_indices(N)
    arr = CuArray{Int}(0:N-1)
    return arr
end

cu_indices (generic function with 1 method)

In [66]:
indices = cu_indices(N)


cu_ktoi(N, indices)

CuError: CUDA error: unspecified launch failure (code 719, ERROR_LAUNCH_FAILED)

In [ ]:
function cu_interleave(block, type=:qpsk)
    N = Dict(:qpsk => 96, :qam16 => 192, :qam64 => 288)[type]
    s = Dict(:qpsk => 1, :qam16 => 2, :qam64 => 3)[type]
    ks = cu_indices(N[type])
    is = cu_ktoi(N[type], ks)
    js = cu_itoj(N[type], s[type], is)
    return block[js]
end

@info "Interleave"
t_interleave = @elapsed begin
    blocks = interleave.(blocks, scheme)
end


In [ ]:
function modulate(bits, bit_map)
    n = length(bits) ÷ 2
    return (bit_map[bits[1:n]] |> Float64) + im * (bit_map[bits[n + 1:end]] |> Float64)
end

function modulate_block(block, type=:qpsk)
    values = Dict(        
        :qpsk  => Dict([0] => -1,
                       [1] => 1),
        :qam16 => Dict([0, 0] => -3, [0, 1] => -1,
                       [1, 1] => 1, [1, 0] => 3),
        :qam64 => Dict([0, 0, 0] => -7, [0, 0, 1] => -5, [0, 1, 1] => -3, [0, 1, 0] => -1,
                       [1, 1, 0] => 1, [1, 1, 1] => 3, [1, 0, 1] => 5, [1, 0, 0] => 7))
    nbits = Dict(:bpsk => 1, :qpsk => 2, :qam16 => 4, :qam64 => 6)
    return [modulate(x, values[type]) for x in Iterators.partition(block, nbits[type])]
end

@info "Modulation"
t_modulation = @elapsed begin
    blocks = modulate_block.(blocks, scheme)
end

In [ ]:
function add_pilots(x)
    return vcat(x[1:5], 0, x[6:18], 0, x[19:24], 0, x[25:30], 0, x[31:43], 0, x[44:48])
end

function shift(x)
    return vcat(x[1:26], zeros(12), x[28:53])
end
    

@info "Pilot Waves"
t_pilot_waves = @elapsed begin
    pblocks = add_pilots.(blocks)
    sblocks = shift.(pblocks)
end

In [ ]:

@info "IFFT"
t_ifft = @elapsed begin
    waves = ifft.(sblocks)
end

In [ ]:
pblock = pblocks[1]
p1 = plot(-26:26, pblock |> real, line=:stem, marker=:square, markersize=3, ylabel=L"\mathcal{Re}", title="Descripción Frecuencia")
p2 = plot(-26:26, pblock |> imag, line=:stem, marker=:square, markersize=3, ylabel=L"\mathcal{Im}", xlabel="k")

plot(p1, p2, layout=(2, 1), legend=false)

savefig("freq.png")

wave = waves[1]

t = range(0, 8, length=length(wave))

p1 = plot(t, wave |> real, line=:steppost, linewidth=1, ylabel="canal I", title="Señal Transmitida")
p2 = plot(t, wave |> imag, line=:steppost, linewidth=1, ylabel="canal Q", xlabel="t [μs]")

plot(p1, p2, layout=(2, 1), legend=false)

savefig("wave.png")
